# Fixed Income Asset Pricing - Complete Solution
## Bus 35130 Spring 2024 - John Heaton

This notebook provides autonomous solutions to all homework assignments (HW1-HW7) for the Fixed Income Asset Pricing course.

### Table of Contents
1. [HW1: Interest Rate Forecasting](#hw1)
2. [HW2: Leveraged Inverse Floaters](#hw2)
3. [HW3: Duration Hedging and Factor Neutrality](#hw3)
4. [HW4: Real and Nominal Bonds](#hw4)
5. [HW5: Caps, Floors, and Swaptions](#hw5)
6. [HW6: Callable Bonds](#hw6)
7. [HW7: MBS and Relative Value Trades](#hw7)

In [ ]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.optimize import minimize, brentq
from scipy.stats import norm
from scipy.interpolate import interp1d, CubicSpline
import xlrd
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 6)

print("All libraries imported successfully!")

## Utility Functions

We'll define helper functions that will be used throughout the assignments.

In [ ]:
# Helper functions for bond pricing and analysis

def discount_to_bey(d, n=91):
    """Convert discount rate to Bond Equivalent Yield"""
    return (365 * d) / (360 - n * d)

def bey_to_discount(r, n=91):
    """Convert Bond Equivalent Yield to discount rate"""
    return (360 * r) / (365 + n * r)

def price_from_yield(ytm, coupon, maturity, freq=2, face=100):
    """Calculate bond price from yield to maturity"""
    n_periods = int(maturity * freq)
    c = coupon * face / freq
    y = ytm / freq
    
    if ytm == 0:
        return face + c * n_periods
    
    pv_coupons = c * (1 - (1 + y)**(-n_periods)) / y
    pv_face = face / (1 + y)**n_periods
    return pv_coupons + pv_face

def yield_from_price(price, coupon, maturity, freq=2, face=100):
    """Calculate yield to maturity from bond price"""
    def price_diff(y):
        return price_from_yield(y, coupon, maturity, freq, face) - price
    
    try:
        return brentq(price_diff, -0.1, 1.0)
    except:
        return np.nan

def duration(ytm, coupon, maturity, freq=2, face=100):
    """Calculate modified duration"""
    n_periods = int(maturity * freq)
    c = coupon * face / freq
    y = ytm / freq
    price = price_from_yield(ytm, coupon, maturity, freq, face)
    
    pv_weighted = 0
    for t in range(1, n_periods + 1):
        if t < n_periods:
            cf = c
        else:
            cf = c + face
        pv_weighted += (t / freq) * cf / (1 + y)**t
    
    macaulay_dur = pv_weighted / price
    modified_dur = macaulay_dur / (1 + y)
    return modified_dur

def convexity(ytm, coupon, maturity, freq=2, face=100):
    """Calculate convexity"""
    n_periods = int(maturity * freq)
    c = coupon * face / freq
    y = ytm / freq
    price = price_from_yield(ytm, coupon, maturity, freq, face)
    
    pv_weighted = 0
    for t in range(1, n_periods + 1):
        if t < n_periods:
            cf = c
        else:
            cf = c + face
        pv_weighted += t * (t + 1) * cf / (1 + y)**(t + 2)
    
    return pv_weighted / price

def nelson_siegel(tau, beta0, beta1, beta2, lambda_param):
    """Nelson-Siegel yield curve model"""
    factor1 = (1 - np.exp(-lambda_param * tau)) / (lambda_param * tau)
    factor2 = factor1 - np.exp(-lambda_param * tau)
    return beta0 + beta1 * factor1 + beta2 * factor2

def fit_nelson_siegel(maturities, yields):
    """Fit Nelson-Siegel model to yield curve data"""
    def objective(params):
        beta0, beta1, beta2, lambda_param = params
        fitted = nelson_siegel(maturities, beta0, beta1, beta2, lambda_param)
        return np.sum((fitted - yields)**2)
    
    # Initial guess
    x0 = [yields[-1], yields[0] - yields[-1], 0, 1]
    bounds = [(None, None), (None, None), (None, None), (0.01, 10)]
    
    result = minimize(objective, x0, bounds=bounds, method='L-BFGS-B')
    return result.x

print("Utility functions defined successfully!")

<a id='hw1'></a>
# Homework 1: Interest Rate Forecasting

## Overview
- Convert discount rates to Bond Equivalent Yield (BEY)
- Estimate AR(1) process for interest rates
- Forecast future interest rates
- Compute forward rates from Treasury Strip prices

In [ ]:
# Load HW1 data
try:
    # Read DTB3 data (3-month T-bill rates)
    dtb3_file = '/home/user/Fixed-Income-Asset-Pricing/Assignments/PSET 1/DTB3_2024.xls'
    
    # Read the Excel file
    wb = xlrd.open_workbook(dtb3_file)
    
    # Read DTB3 sheet
    sheet_dtb3 = wb.sheet_by_name('DTB3')
    dates = []
    rates = []
    
    for i in range(1, sheet_dtb3.nrows):  # Skip header
        try:
            date_val = sheet_dtb3.cell_value(i, 0)
            rate_val = sheet_dtb3.cell_value(i, 1)
            if rate_val != '' and str(rate_val).upper() != 'ND':
                dates.append(pd.to_datetime(xlrd.xldate_as_datetime(date_val, wb.datemode)))
                rates.append(float(rate_val))
        except:
            continue
    
    dtb3_data = pd.DataFrame({'Date': dates, 'Discount_Rate': rates})
    dtb3_data.set_index('Date', inplace=True)
    
    # Read Strip Prices sheet
    sheet_strips = wb.sheet_by_name('Strip Prices')
    strip_maturities = []
    strip_prices = []
    
    for i in range(1, sheet_strips.nrows):  # Skip header
        try:
            mat_val = sheet_strips.cell_value(i, 0)
            price_val = sheet_strips.cell_value(i, 1)
            if mat_val != '' and price_val != '':
                strip_maturities.append(float(mat_val))
                strip_prices.append(float(price_val))
        except:
            continue
    
    strip_data = pd.DataFrame({'Maturity': strip_maturities, 'Price': strip_prices})
    
    print(f"Loaded {len(dtb3_data)} observations of 3-month T-bill rates")
    print(f"Date range: {dtb3_data.index[0]} to {dtb3_data.index[-1]}")
    print(f"\nLoaded {len(strip_data)} Treasury Strip prices")
    
    # Display first few rows
    print("\nFirst few T-bill observations:")
    display(dtb3_data.head(10))
    
    print("\nTreasury Strip Prices:")
    display(strip_data.head(10))
    
except Exception as e:
    print(f"Error loading HW1 data: {e}")
    import traceback
    traceback.print_exc()

### Question 1: Convert Discount Rates to Bond Equivalent Yield

The Bond Equivalent Yield (BEY) formula is:
$$r_{BEY} = \frac{365 \times d}{360 - n \times d}$$

where $d$ is the discount rate and $n = 91$ days for 3-month T-bills.

In [ ]:
# Convert discount rates to BEY
dtb3_data['BEY'] = dtb3_data['Discount_Rate'].apply(lambda d: discount_to_bey(d/100, n=91) * 100)

# Plot the time series
fig = go.Figure()
fig.add_trace(go.Scatter(x=dtb3_data.index, y=dtb3_data['Discount_Rate'], 
                         mode='lines', name='Discount Rate'))
fig.add_trace(go.Scatter(x=dtb3_data.index, y=dtb3_data['BEY'], 
                         mode='lines', name='Bond Equivalent Yield'))

fig.update_layout(
    title='3-Month T-Bill: Discount Rate vs Bond Equivalent Yield',
    xaxis_title='Date',
    yaxis_title='Rate (%)',
    hovermode='x unified',
    template='plotly_white',
    height=500
)

fig.show()

print("\nSummary Statistics:")
print(dtb3_data[['Discount_Rate', 'BEY']].describe())

### Question 2: AR(1) Process Estimation

The AR(1) model is:
$$r_{t+1} = \alpha + \beta r_t + \epsilon_{t+1}$$

where $\epsilon_{t+1} \sim N(0, \sigma^2)$

**OLS Formulas:**
$$\hat{\beta} = \frac{\text{Cov}(r_t, r_{t+1})}{\text{Var}(r_t)}$$
$$\hat{\alpha} = \bar{r}_{t+1} - \hat{\beta}\bar{r}_t$$
$$\hat{\sigma}^2 = \frac{1}{n-2}\sum_{t=1}^{n-1}(r_{t+1} - \hat{\alpha} - \hat{\beta}r_t)^2$$

**Mean Reversion:** When $0 < \beta < 1$, the process exhibits mean reversion. The long-run mean is $\frac{\alpha}{1-\beta}$.

In [ ]:
# Prepare data for AR(1) estimation
r_t = dtb3_data['BEY'].values[:-1]
r_t_plus_1 = dtb3_data['BEY'].values[1:]

# OLS estimation
beta_hat = np.cov(r_t, r_t_plus_1)[0,1] / np.var(r_t)
alpha_hat = np.mean(r_t_plus_1) - beta_hat * np.mean(r_t)

# Residuals
residuals = r_t_plus_1 - alpha_hat - beta_hat * r_t
sigma_hat = np.std(residuals, ddof=2)

# Long-run mean
long_run_mean = alpha_hat / (1 - beta_hat)

print("AR(1) Model Estimation Results:")
print("="*50)
print(f"α̂ (alpha): {alpha_hat:.6f}")
print(f"β̂ (beta):  {beta_hat:.6f}")
print(f"σ̂ (sigma): {sigma_hat:.6f}")
print(f"\nLong-run mean: {long_run_mean:.4f}%")
print(f"\nMean reversion: {'Yes' if 0 < beta_hat < 1 else 'No'}")
print(f"Half-life (days): {-np.log(2)/np.log(beta_hat):.1f}" if 0 < beta_hat < 1 else "N/A")

# Diagnostic plots
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=('Actual vs Fitted', 'Residuals Over Time', 
                                   'Residual Histogram', 'Actual vs Lagged'))

# Actual vs Fitted
fitted = alpha_hat + beta_hat * r_t
fig.add_trace(go.Scatter(y=r_t_plus_1, mode='markers', name='Actual', 
                         marker=dict(size=3, opacity=0.5)), row=1, col=1)
fig.add_trace(go.Scatter(y=fitted, mode='markers', name='Fitted',
                         marker=dict(size=3, opacity=0.5)), row=1, col=1)

# Residuals over time
fig.add_trace(go.Scatter(y=residuals, mode='lines', name='Residuals',
                         line=dict(width=1)), row=1, col=2)
fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=2)

# Residual histogram
fig.add_trace(go.Histogram(x=residuals, name='Residual Distribution',
                           nbinsx=50), row=2, col=1)

# Scatter plot: r_t+1 vs r_t
fig.add_trace(go.Scatter(x=r_t, y=r_t_plus_1, mode='markers', name='Data',
                         marker=dict(size=3, opacity=0.3)), row=2, col=2)
# Add regression line
r_range = np.array([r_t.min(), r_t.max()])
fig.add_trace(go.Scatter(x=r_range, y=alpha_hat + beta_hat*r_range, 
                         mode='lines', name='AR(1) Fit', line=dict(color='red')), 
              row=2, col=2)

fig.update_layout(height=800, showlegend=True, template='plotly_white',
                 title_text="AR(1) Model Diagnostics")
fig.show()

### Question 3: Forecasting Future Interest Rates

Using the AR(1) model, we forecast:
$$\hat{r}_{T+h} = \hat{\alpha}\sum_{i=0}^{h-1}\hat{\beta}^i + \hat{\beta}^h r_T$$

For long horizons, as $h \to \infty$:
$$\hat{r}_{T+h} \to \frac{\hat{\alpha}}{1-\hat{\beta}}$$ (the long-run mean)

In [ ]:
# Get the most recent rate
r_today = dtb3_data['BEY'].iloc[-1]
print(f"Most recent BEY (as of {dtb3_data.index[-1].date()}): {r_today:.4f}%")

# Forecast for next 3 days (pencil and paper requirement)
print("\n3-Day Forecast (Pencil and Paper):")
print("="*50)
for day in range(1, 4):
    if day == 1:
        forecast = alpha_hat + beta_hat * r_today
    else:
        forecast = alpha_hat + beta_hat * forecast
    print(f"Day {day}: r̂ = {alpha_hat:.6f} + {beta_hat:.6f} × {forecast if day > 1 else r_today:.4f} = {forecast:.4f}%")

# Forecast for 6 months, 1-5 years
days_per_year = 252  # Business days
horizons_days = [days_per_year//2] + [days_per_year * i for i in range(1, 6)]
horizons_labels = ['6 months', '1 year', '2 years', '3 years', '4 years', '5 years']

forecasts = []
for h in horizons_days:
    # Forecast formula: alpha * sum(beta^i, i=0 to h-1) + beta^h * r_today
    if abs(beta_hat - 1) > 1e-10:
        forecast = alpha_hat * (1 - beta_hat**h) / (1 - beta_hat) + (beta_hat**h) * r_today
    else:
        forecast = alpha_hat * h + r_today
    forecasts.append(forecast)

# Create forecast dataframe
forecast_df = pd.DataFrame({
    'Horizon': horizons_labels,
    'Days': horizons_days,
    'Forecast (%)': forecasts,
    'Long-run Mean (%)': [long_run_mean] * len(horizons_labels)
})

print("\nLong-Horizon Forecasts:")
display(forecast_df)

# Plot forecasts
all_days = list(range(0, max(horizons_days) + 1))
all_forecasts = []
for h in all_days:
    if h == 0:
        all_forecasts.append(r_today)
    else:
        if abs(beta_hat - 1) > 1e-10:
            fc = alpha_hat * (1 - beta_hat**h) / (1 - beta_hat) + (beta_hat**h) * r_today
        else:
            fc = alpha_hat * h + r_today
        all_forecasts.append(fc)

fig = go.Figure()
fig.add_trace(go.Scatter(x=all_days, y=all_forecasts, mode='lines', name='AR(1) Forecast'))
fig.add_hline(y=long_run_mean, line_dash="dash", line_color="red", 
              annotation_text=f"Long-run mean: {long_run_mean:.2f}%")
fig.add_scatter(x=horizons_days, y=forecasts, mode='markers', name='Key Horizons',
                marker=dict(size=10, color='orange'))

fig.update_layout(
    title='Interest Rate Forecasts from AR(1) Model',
    xaxis_title='Days Ahead',
    yaxis_title='Forecasted Rate (%)',
    template='plotly_white',
    height=500
)
fig.show()

### Question 4: Forward Rates from Treasury Strips

**Spot Rate from Price:**
$$r(0,T) = -\frac{\ln(Z(0,T))}{T}$$

where $Z(0,T)$ is the zero-coupon bond price.

**Forward Rate:**
The forward rate $f(T_1, T_2)$ is the rate for borrowing/lending between time $T_1$ and $T_2$:
$$f(T_1,T_2) = \frac{r(0,T_2) \cdot T_2 - r(0,T_1) \cdot T_1}{T_2 - T_1}$$

Instantaneous forward rate:
$$f(0,T) = r(0,T) + T \cdot \frac{\partial r(0,T)}{\partial T}$$

In [ ]:
# Calculate spot rates from strip prices
strip_data['Spot_Rate'] = -np.log(strip_data['Price'] / 100) / strip_data['Maturity'] * 100

print("Treasury Strip Prices and Spot Rates:")
print("="*60)

# Show first 3 maturities with detailed calculations
print("\nDetailed Calculations for First Three Maturities:")
for i in range(min(3, len(strip_data))):
    T = strip_data.iloc[i]['Maturity']
    P = strip_data.iloc[i]['Price']
    r = strip_data.iloc[i]['Spot_Rate']
    print(f"\nMaturity T = {T:.2f} years")
    print(f"  Price Z(0,{T:.2f}) = ${P:.4f}")
    print(f"  Spot Rate: r(0,{T:.2f}) = -ln({P}/100)/{T:.2f} = {r:.4f}%")

display(strip_data.head(10))

# Calculate forward rates
forward_rates = []
forward_labels = []

for i in range(len(strip_data) - 1):
    T1 = strip_data.iloc[i]['Maturity']
    T2 = strip_data.iloc[i+1]['Maturity']
    r1 = strip_data.iloc[i]['Spot_Rate'] / 100
    r2 = strip_data.iloc[i+1]['Spot_Rate'] / 100
    
    # Forward rate between T1 and T2
    f = ((r2 * T2 - r1 * T1) / (T2 - T1)) * 100
    forward_rates.append(f)
    forward_labels.append(f"{T1:.1f}-{T2:.1f}y")

# Show forward rate calculations for first 3
print("\n\nForward Rate Calculations (First Three):")
print("="*60)
for i in range(min(3, len(forward_rates))):
    T1 = strip_data.iloc[i]['Maturity']
    T2 = strip_data.iloc[i+1]['Maturity']
    r1 = strip_data.iloc[i]['Spot_Rate']
    r2 = strip_data.iloc[i+1]['Spot_Rate']
    f = forward_rates[i]
    print(f"\nForward rate f({T1:.2f},{T2:.2f}):")
    print(f"  f = [r(0,{T2:.2f})×{T2:.2f} - r(0,{T1:.2f})×{T1:.2f}] / ({T2:.2f}-{T1:.2f})")
    print(f"  f = [{r2:.4f}×{T2:.2f} - {r1:.4f}×{T1:.2f}] / {T2-T1:.2f}")
    print(f"  f = {f:.4f}%")

# Plot yield curve and forward rates
fig = make_subplots(rows=1, cols=2, subplot_titles=('Spot Rate Curve', 'Forward Rate Curve'))

fig.add_trace(go.Scatter(x=strip_data['Maturity'], y=strip_data['Spot_Rate'],
                         mode='lines+markers', name='Spot Rates'), row=1, col=1)

fig.add_trace(go.Scatter(x=strip_data['Maturity'].iloc[1:], y=forward_rates,
                         mode='lines+markers', name='Forward Rates'), row=1, col=2)

fig.update_xaxes(title_text="Maturity (years)", row=1, col=1)
fig.update_xaxes(title_text="Maturity (years)", row=1, col=2)
fig.update_yaxes(title_text="Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Rate (%)", row=1, col=2)

fig.update_layout(height=500, template='plotly_white',
                 title_text="Treasury Yield Curve Analysis (March 8, 2024)")
fig.show()

# Compare AR(1) forecasts with forward rates
print("\n\nComparison: AR(1) Forecasts vs Forward Rates")
print("="*60)
comparison_df = pd.DataFrame({
    'Horizon': horizons_labels,
    'AR(1) Forecast (%)': forecasts
})

# Match forward rates to horizons (approximately)
forward_at_horizons = []
for h_days in horizons_days:
    h_years = h_days / 252
    # Find closest maturity in strip data
    idx = np.argmin(np.abs(strip_data['Maturity'].values - h_years))
    if idx < len(forward_rates):
        forward_at_horizons.append(forward_rates[idx])
    else:
        forward_at_horizons.append(forward_rates[-1])

comparison_df['Forward Rate (%)'] = forward_at_horizons
comparison_df['Difference (%)'] = comparison_df['AR(1) Forecast (%)'] - comparison_df['Forward Rate (%)']

display(comparison_df)

print("\n" + "="*60)
print("HOMEWORK 1 COMPLETED SUCCESSFULLY!")
print("="*60)